In [ ]:
import warnings
import sys
import os

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [ ]:
from src.main.models.transformers import Transformer
from config import config
from src.main.pipeline.functions import clean_text, stop_words_removal
from src.main.utilities.utils import get_dataset, split_train_val_test
import numpy as np

inputs, targets = get_dataset(one_hot=False)

checkpoint = "textattack/bert-base-uncased-ag-news"

training_args = {
    'output_dir': config.RESULTS_DIRECTORY.format(checkpoint),
    'num_train_epochs': 5,
    'per_device_train_batch_size': 16,
    'per_device_eval_batch_size': 16,
    'learning_rate': 2e-5,
    'warmup_steps': 500,
    'weight_decay': 0.01,
    'logging_dir': config.LOGS_PATH.format(checkpoint),
    'logging_steps': 500,
    'evaluation_strategy': 'epoch',
    'save_strategy': 'epoch',
    'load_best_model_at_end': True,
    'metric_for_best_model': 'accuracy'
}

distilBert = Transformer(checkpoint=checkpoint, **training_args)
distilBert.pipeline = [clean_text, stop_words_removal]
inputs = distilBert.run_pipeline(inputs, save=True)
targets = np.vectorize(config.label2id.get)(targets)

x_train, x_val, x_test, y_train, y_val, y_test = split_train_val_test(inputs, targets, validation_size=0.2, test_size=0.1)

distilBert.prepare_dataset(x_train, x_val, y_train, y_val)
distilBert.fit(x_train, y_train)

test_dataset = distilBert.prepare_test_dataset(x_test, y_test)
distilBert.save_results(test_dataset)
distilBert.save_model()